In [1]:
import chess
import numpy as np
import pandas as pd


In [2]:
def disp(fen):
    board = chess.Board(fen)
    return board


In [3]:
abs_path="d:\OneDrive - Indian Institute of Technology Bombay\Study\Events and Activities\Year 2\Seasons of Coding 23\soc23-DeepCarlsen"
TotalPos = pd.read_csv(abs_path+"\dataset\chessDataEval.csv")


In [30]:
def cond(arr):
    pos = arr["FEN"].split()[0]
    res = pos.count('Q')>1 or pos.count('B')>2 or pos.count('N')>2 or pos.count('R')>2 or pos.count('q')>1 or pos.count('b')>2 or pos.count('n')>2 or pos.count('r')>2
    return not res


In [46]:
dataset_raw = TotalPos[TotalPos.apply(cond, axis = 1)]


In [47]:
dataset1 = dataset_raw.sample(100000)

In [48]:
dataset = dataset1#.sample(1000)
dataset.reset_index(inplace=True)
dataset = dataset.drop(["index","level_0"], axis=1)
dataset

,FEN,Evaluation
0,r4rk1/p3ppbp/1p4p1/3b4/6P1/PN2B2P/1PP1BP2/2KR4...,+352
1,r1b1k2r/ppp1qppp/2pb1n2/4p3/4P3/3P1N1P/PPPNQPP...,0
2,5r1k/pp2b1pp/2p1qnn1/4p3/N2r4/P5PP/1PQBPPB1/3R...,+109
3,1rb2rk1/p1p1qpp1/1p6/4P1p1/3pN3/3P2PP/PP2Q1P1/...,-80
4,8/ppk5/8/1P2Rp1p/P6P/5KP1/r7/8 b - - 4 39,+118
...,...,...
99995,8/1R2p2p/3k4/3p2p1/8/4RPP1/r3b2P/6K1 w - - 4 34,+129
99996,1k2r3/4r1p1/1pQ1pp1p/q5PP/1ppP1P2/8/PPP5/1K1RR...,+13
99997,8/3n2K1/3k4/5P2/3B2N1/8/8/8 w - - 17 62,+6362
99998,r4rk1/1b2B1bp/pn1pp1p1/1p6/4P3/P1NP1N1P/1P4P1/...,0


In [5]:
def features_global(arr):
    fen = arr if isinstance(arr, str) else arr["FEN"]
    board = chess.Board(fen)
    white_play = (board.turn==chess.WHITE)
    K = board.has_kingside_castling_rights(chess.WHITE)
    k = board.has_kingside_castling_rights(chess.BLACK)
    Q = board.has_queenside_castling_rights(chess.WHITE)
    q = board.has_queenside_castling_rights(chess.BLACK)
    return [int(white_play),int(K), int(k), int(Q), int(q)]

In [6]:
def features_piece(arr):
    def p(symbol):
        return chess.Piece.from_symbol(symbol)
    fen = arr if isinstance(arr, str) else arr["FEN"]
    board = chess.Board(fen)
    pieces_map = chess.Board.piece_map(board)
    emp = [-1,-1]
    piece_num = [0]*14
    piece_w = [emp]*16  # K Q B(light) B(dark) N N R R P P P P P P P P
    piece_b = [emp]*16  # k q b(light) b(dark) n n r r p p p p p p p p
    for (loc, pi) in pieces_map.items():
        if pi== p('k'):
            piece_b[0] = [loc%8,loc//8]
            piece_num[7]+=1
        elif pi== p('q'):
            piece_b[1] = [loc%8,loc//8]
            piece_num[8]+=1
        elif pi== p('b'):
            if((loc%8+loc//8)%2!=0):
                piece_b[2] = [loc%8,loc//8]
                piece_num[9]+=1
            else:
                piece_b[3] = [loc%8,loc//8]
                piece_num[10]+=1
        elif pi== p('n'):
            piece_b[piece_b[4:6].index(emp)+4]=[loc%8,loc//8]
            piece_num[11]+=1
        elif pi== p('r'):
            piece_b[piece_b[6:8].index(emp)+6]=[loc%8,loc//8]
            piece_num[12]+=1
        elif pi== p('p'):
            piece_b[piece_b[8:16].index(emp)+8]=[loc%8,loc//8]
            piece_num[13]+=1

        elif pi== p('K'):
            piece_w[0] = [loc%8,loc//8]
            piece_num[0]+=1
        elif pi== p('Q'):
            piece_w[1] = [loc%8,loc//8]
            piece_num[1]+=1
        elif pi== p('B'):
            if((loc%8+loc//8)%2!=0):
                piece_w[2] = [loc%8,loc//8]
                piece_num[2]+=1
            else:
                piece_w[3] = [loc%8,loc//8]
                piece_num[3]+=1
        elif pi== p('N'):
            piece_w[piece_w[4:6].index(emp)+4]=[loc%8,loc//8]
            piece_num[4]+=1
        elif pi== p('R'):
            piece_w[piece_w[6:8].index(emp)+6]=[loc%8,loc//8]
            piece_num[5]+=1
        elif pi== p('P'):
            piece_w[piece_w[8:16].index(emp)+8]=[loc%8,loc//8]
            piece_num[6]+=1
    result = piece_num+[item for sublist in piece_w for item in sublist]+[item for sublist in piece_b for item in sublist]
    return result


In [7]:
def features_map(arr):
    fen = arr if isinstance(arr, str) else arr["FEN"]
    piece_value = {6:100, 5:9, 3:3, 2:3, 4:5, 1:1}
    board = chess.Board(fen)
    wht = []
    wht_count = []
    blk = []
    blk_count = []
    for sq in chess.SQUARES:   
        attks_white = chess.BaseBoard.attackers(board, chess.WHITE, sq)
        attks_black = chess.BaseBoard.attackers(board, chess.BLACK, sq)
        wht.append(0 if len(attks_white)==0 else min(piece_value[chess.BaseBoard.piece_type_at(board, sq2)] for sq2 in attks_white))
        blk.append(0 if len(attks_black)==0 else min(piece_value[chess.BaseBoard.piece_type_at(board, sq2)] for sq2 in attks_black))
        wht_count.append(len(attks_white))
        blk_count.append(len(attks_black))
    return wht_count+wht+blk_count+blk

In [8]:
arr = []
fen = "2q1rrk1/1b2ppbp/p2p1np1/P1pP4/2B1PP2/2NQ3P/1P1B2P1/R4RK1 w - - 5 22"
board = chess.Board(fen)
for sq in chess.SQUARES: 
    test = chess.BaseBoard.piece_type_at(board, sq)
    arr.append(0 if test==None else test)
arr

[4,
 0,
 0,
 0,
 0,
 4,
 6,
 0,
 0,
 1,
 0,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 5,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 0,
 3,
 0,
 0,
 1,
 1,
 3,
 1,
 0,
 0,
 5,
 0,
 4,
 4,
 6,
 0]

In [9]:
def feature_extract(evalPos):
        feat_centric = evalPos.copy().drop(["FEN", "Evaluation"], axis=1)
        feat_global = evalPos.copy().drop(["FEN", "Evaluation"], axis=1)
        col_centric = [ "cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp", 
                'wk_col', 'wk_row', 'wq_col', 'wq_row', 'wbl_col', 'wbl_row', 'wbd_col', 'wbd_row', 'wn1_col', 'wn1_row', 'wn2_col', 'wn2_row', 'wr1_col', 'wr1_row', 'wr2_col', 'wr2_row',
                 'wp1_col', 'wp1_row', 'wp2_col', 'wp2_row', 'wp3_col', 'wp3_row', 'wp4_col', 'wp4_row', 'wp5_col', 'wp5_row', 'wp6_col', 'wp6_row', 'wp7_col', 'wp7_row', 'wp8_col', 'wp8_row',
                'bk_col', 'bk_row', 'bq_col', 'bq_row', 'bbl_col', 'bbl_row', 'bbd_col', 'bbd_row', 'bn1_col', 'bn1_row', 'bn2_col', 'bn2_row', 'br1_col', 'br1_row', 'br2_col', 'br2_row', 
                'bp1_col', 'bp1_row', 'bp2_col', 'bp2_row', 'bp3_col', 'bp3_row', 'bp4_col', 'bp4_row', 'bp5_col', 'bp5_row', 'bp6_col', 'bp6_row', 'bp7_col', 'bp7_row', 'bp8_col', 'bp8_row']
        col_global = ["white_turn", "castle_wk", "castle_bk", "castle_wq", "castle_bq"]
        col_transfer = ["cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                        "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp"]
        y = evalPos["Evaluation"]
        feat_centric[col_centric] = evalPos[["FEN"]].apply(features_piece, axis=1, result_type="expand")
        feat_global[col_global] = evalPos[["FEN"]].apply(features_global, axis=1, result_type="expand")
        feat_global = feat_global.join(feat_centric[col_transfer])
        feat_centric.drop(["cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp"], axis=1, inplace=True)
        
        feat_map = evalPos.copy().drop(["FEN", "Evaluation"], axis=1)
        col_map = ["wht_count"+str(i) for i in range(64)]+["wht_attack"+str(i) for i in range(64)]+["blk_count"+str(i) for i in range(64)]+["blk_attack"+str(i) for i in range(64)]
        feat_map[col_map] = evalPos[["FEN"]].apply(features_map, axis=1, result_type="expand")
        feat_map
        return feat_global, feat_centric, feat_map, y

In [10]:
col_centric = [ "cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp", 
                'wk_col', 'wk_row', 'wq_col', 'wq_row', 'wbl_col', 'wbl_row', 'wbd_col', 'wbd_row', 'wn1_col', 'wn1_row', 'wn2_col', 'wn2_row', 'wr1_col', 'wr1_row', 'wr2_col', 'wr2_row',
                 'wp1_col', 'wp1_row', 'wp2_col', 'wp2_row', 'wp3_col', 'wp3_row', 'wp4_col', 'wp4_row', 'wp5_col', 'wp5_row', 'wp6_col', 'wp6_row', 'wp7_col', 'wp7_row', 'wp8_col', 'wp8_row',
                'bk_col', 'bk_row', 'bq_col', 'bq_row', 'bbl_col', 'bbl_row', 'bbd_col', 'bbd_row', 'bn1_col', 'bn1_row', 'bn2_col', 'bn2_row', 'br1_col', 'br1_row', 'br2_col', 'br2_row', 
                'bp1_col', 'bp1_row', 'bp2_col', 'bp2_row', 'bp3_col', 'bp3_row', 'bp4_col', 'bp4_row', 'bp5_col', 'bp5_row', 'bp6_col', 'bp6_row', 'bp7_col', 'bp7_row', 'bp8_col', 'bp8_row']
print(len(col_centric))
fen = "2q1rrk1/1b2ppbp/p2p1np1/P1pP4/2B1PP2/2NQ3P/1P1B2P1/R4RK1 w - - 5 22"
board = chess.Board(fen)
ar = features_piece(fen)
print(len(ar))

78
78


In [11]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense

In [12]:
from tensorflow.keras.optimizers import Adam
import math
from tensorflow.keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [49]:
feat1, feat2, feat3, y_raw = feature_extract(dataset)

C:\Users\spand\AppData\Local\Temp\ipykernel_37680\1613683386.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat_map[col_map] = evalPos[["FEN"]].apply(features_map, axis=1, result_type="expand")
C:\Users\spand\AppData\Local\Temp\ipykernel_37680\1613683386.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat_map[col_map] = evalPos[["FEN"]].apply(features_map, axis=1, result_type="expand")
C:\Users\spand\AppData\Local\Temp\ipykernel_37680\1613683386.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usu

In [50]:
x = pd.concat([feat1, feat2, feat3], axis=1)
y = y_raw.apply(lambda l: int(l) if '#' not in l else +999 if '+' in l else -999)
x.describe()

,white_turn,castle_wk,castle_bk,castle_wq,castle_bq,cwk,cwq,cwbl,cwbd,cwn,...,blk_attack54,blk_attack55,blk_attack56,blk_attack57,blk_attack58,blk_attack59,blk_attack60,blk_attack61,blk_attack62,blk_attack63
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.0,100000.000000,100000.000000,100000.00000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500620,0.052860,0.059310,0.047420,0.051950,1.0,0.630200,0.478210,0.48940,0.890030,...,46.895160,42.349750,4.176300,5.347820,5.567360,8.18113,8.463730,15.720950,12.819400,36.744900
std,0.500002,0.223755,0.236205,0.212537,0.221927,0.0,0.482753,0.499527,0.49989,0.770183,...,48.460648,47.995287,11.491464,14.069422,14.288685,20.77445,21.551745,32.225627,28.814547,46.652687
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.00000,0.000000,...,3.000000,3.000000,0.000000,0.000000,3.000000,3.00000,3.000000,3.000000,3.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,0.00000,1.000000,...,9.000000,5.000000,3.000000,3.000000,5.000000,5.00000,3.000000,3.000000,5.000000,5.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,1.000000,1.00000,1.000000,...,100.000000,100.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,100.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.00000,2.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000


In [ ]:
tf.convert_to_tensor(np.asarray(x).astype(np.float_))

C:\Users\spand\AppData\Local\Temp\ipykernel_24644\159561658.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tf.convert_to_tensor(np.asarray(x).astype(np.float))


<tf.Tensor: shape=(1000, 339), dtype=float64, numpy=
array([[  1.,   0.,   0., ...,   3.,   5., 100.],
       [  1.,   0.,   0., ...,   5.,   5., 100.],
       [  0.,   0.,   0., ...,   0., 100.,   0.],
       ...,
       [  1.,   0.,   0., ...,   3.,   9.,   9.],
       [  1.,   0.,   0., ...,   3.,   0., 100.],
       [  1.,   0.,   0., ...,   5.,   5., 100.]])>

In [41]:
def tens(arra):
    return tf.convert_to_tensor(np.asarray(arra).astype(np.float_))

In [51]:
model = tf.keras.Sequential([Dense(512, activation="relu"), Dense(128, activation="relu"), Dense(1)])
model.compile(loss=rmse, optimizer=Adam(), metrics=[rmse])


In [52]:
nlim = int(len(x)*.8)
model.fit(tens(x[:nlim]), tens(y[:nlim]), epochs=10 )

Epoch 1/10
2500/2500 [==============================] - 12s 2ms/step - loss: 668.9927 - rmse: 668.9927
Epoch 2/10
2500/2500 [==============================] - 6s 2ms/step - loss: 638.0134 - rmse: 638.0134
Epoch 3/10
2500/2500 [==============================] - 5s 2ms/step - loss: 621.4274 - rmse: 621.4274
Epoch 4/10
2500/2500 [==============================] - 5s 2ms/step - loss: 607.4617 - rmse: 607.4617
Epoch 5/10
2500/2500 [==============================] - 5s 2ms/step - loss: 594.6392 - rmse: 594.6392
Epoch 6/10
2500/2500 [==============================] - 5s 2ms/step - loss: 575.0889 - rmse: 575.0889
Epoch 7/10
2500/2500 [==============================] - 5s 2ms/step - loss: 556.6763 - rmse: 556.6763
Epoch 8/10
2500/2500 [==============================] - 5s 2ms/step - loss: 534.7460 - rmse: 534.7460
Epoch 9/10
2500/2500 [==============================] - 5s 2ms/step - loss: 515.0792 - rmse: 515.0792
Epoch 10/10
2500/2500 [==============================] - 5s 2ms/step - loss: 495.

In [53]:
y_pred = model.predict(tens(x[nlim:]))
rmse(y_pred.flatten(), y[nlim:])

625/625 [==============================] - 1s 918us/step


<tf.Tensor: shape=(), dtype=float64, numpy=738.3729518522872>

In [54]:
df = pd.DataFrame({'Actual': y[nlim:], 'Predicted': y_pred.flatten()})
df

,Actual,Predicted
80000,87,64.841705
80001,0,1.315298
80002,0,10.080529
80003,-210,97.557182
80004,-113,-346.447601
...,...,...
99995,129,1441.841919
99996,13,196.349930
99997,6362,1193.747437
99998,0,16.472631
